# Consensus K-Means

In [16]:
import pandas as pd
import numpy as np
# from sklearn.cluster import KMeans
import os
os.chdir("..") if "notebook" in os.getcwd() else None
import config
from tqdm import tqdm
# np.random.seed(42)

# Load data
X = pd.read_csv(os.path.join(config.DATA_FOLDER, 'ecoli.csv'))
X = X.sample(frac=1).reset_index(drop=True)
y = X.pop('target')
X = X.fillna(X.mean())
# X = (X - X.mean()) / X.std()
X = X.values
# X[-1, :] = (X[-1, :] - X[-1, :].min())/ (X[-1, :].max() - X[-1, :].min()) * 100

In [17]:
import numpy as np
import numpy_indexed as npi
from scipy.spatial.distance import cdist
import config
from scipy.stats.contingency import crosstab
from tqdm import tqdm
from scipy.stats import entropy

class ConsensusKMeans:

    r = config.R

    def __init__(self, n_clusters, cluster_sizes, n_init=10, max_iter=1000, tol=1e-10, 
                 random_state=None, type='Uc', p=5, normalize=False):
        self.k = n_clusters
        # self.distance = distance # 'euclidean', 'jensenshannon', 'cosine', 'p'
        self.n_init = n_init
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.centroids = None
        self.labels_ = None
        self.cluster_sizes = cluster_sizes
        self.ls_partitions_labels = []
        self.type = type
        self.normalize = normalize
        self.p = p
        # self.weights = weights if weights is not None else np.ones(self.r) / self.r

        if type == 'Uc':
            self.d_dist = {'metric': 'euclidean'}
        elif type == 'Ucos':
            self.d_dist = {'metric': 'cosine'}
        elif type == 'ULp':
            self.d_dist = {'metric': 'minkowski', 'p': p}

    def initialize_centroids(self, X):
        X_unique = np.unique(X, axis=0)
        idxs = np.random.choice(X_unique.shape[0], self.k)
        return X_unique[idxs, :]
    
    @staticmethod
    def extract_p(pi, pi_i):
        '''
        The information that we need to extract from the contingency table is:
            - $p_{kj}^{(i)}$ = the ratio of objects in consensus cluster k that belong to cluster j based on partition i
            - $p_k$ = the number of objects in consensus cluster k (p_k+)
            - $P_k^{i}$ = $p_{kj}^{(i)}/p_k$
            - $P^(i) = the vector with the ratios of objects in each cluster based on partition i
        '''
        # Calculate contingency matrix (n)
        cont_i = crosstab(pi, pi_i).count

        # Get p-contingency matrix
        p_cont_i = cont_i / cont_i.sum()

        # Calculate the p_k for each cluster in PI
        p_k = p_cont_i.sum(axis=1)
        p_k = p_k.reshape(-1, 1)

        # Get the P_k^(i)
        P_k_i = p_cont_i / p_k

        # This is a constant and it's the ratio of points in each cluster in pi_i
        P_i = p_cont_i.sum(axis=0)

        return p_k, P_k_i, P_i
    
    def update_centroids(self, centroids_old, labels):
        centroids = centroids_old.copy()
        for j, k in enumerate(np.unique(labels)):
            ls_i = []
            for pi_i in self.ls_partitions_labels:
                _, P_k_i, _ = self.extract_p(labels, pi_i)
                # print(P_k_i.shape)
                ls_i.extend(P_k_i[j, :])
            centroids[k, :] = ls_i
            
        return centroids
    
    @staticmethod
    def compute_kl_divergence_matrix(data, centroids, eps=1E-10):
        """
        Compute KL divergence matrix using broadcasting.

        Parameters:
        - data: Input data array with shape (m, n)
        - centroids: Centroids array with shape (k, n)
        - eps: Small value added to data and centroids to avoid log(0)

        Returns:
        - kl_divergence_matrix: Resulting KL divergence matrix with shape (m, k)
        """
        expanded_data = np.expand_dims(data + eps, axis=1)
        expanded_centroids = np.expand_dims(centroids + eps, axis=0)

        kl_divergence_matrix = np.sum(expanded_data * (np.log2(expanded_data / expanded_centroids)), axis=2)

        return kl_divergence_matrix
    
    def f(self, X_b, centroids, cluster_sizes, weights):

        X_b_split = np.split(X_b, np.cumsum(cluster_sizes), axis=1)[:-1]
        centroids_split = np.split(centroids, np.cumsum(cluster_sizes), axis=1)[:-1]
        eps = 1E-8
        pw = 1
        if self.type == 'Uc':
            pw = 2

        if self.type != 'Uh':
            if self.normalize:
                weight_norm = 1/(self.get_weight_norm(X_b_split, centroids_split) + eps)
                X_r = [cdist(X_b_split[i], centroids_split[i], **self.d_dist) ** pw * weight_norm[i] for i in range(len(X_b_split))]
            else:
                X_r = [cdist(X_b_split[i], centroids_split[i], **self.d_dist) for i in range(len(X_b_split))]
        else:
            X_r = []
            if self.normalize:
                weight_norm = 1/(self.get_weight_norm(X_b_split, centroids_split) + eps)
                X_r = [self.compute_kl_divergence_matrix(X_b_split[i], centroids_split[i], eps=eps) * weight_norm[i] for i in range(len(X_b_split))]
            else:
                X_r = [self.compute_kl_divergence_matrix(X_b_split[i], centroids_split[i], eps=eps) for i in range(len(X_b_split))]

        return np.average(X_r, weights=weights, axis=0)

    def fit(self, X_b, weights=None):

        cluster_sizes = self.cluster_sizes
        X_b_split = np.split(X_b, np.cumsum(cluster_sizes), axis=1)[:-1]
        self.ls_partitions_labels = [np.argmax(x, axis=1) for x in X_b_split]

        if weights is None:
            weights = np.ones(self.r) / self.r
        
        best_inertia = float('inf')
        best_centroids = None
        best_labels = None

        for _ in tqdm(range(self.n_init)):
            if self.random_state is not None:
                np.random.seed(self.random_state)

            centroids = self.initialize_centroids(X_b)
            # labels = np.random.choice(self.k, len(X_b))
            it = 0
            inertia_old = 1E8
            while True:
                d_matrix = self.f(X_b, centroids, self.cluster_sizes, weights=weights)
                labels = d_matrix.argmin(axis=1)
                centroids_old = centroids
                centroids = self.update_centroids(centroids_old, labels)
                it += 1

                inertia = d_matrix.min(axis=1).sum()
                if (inertia_old - inertia) < self.tol:
                    # print(f"Converged in {it} iterations")
                    break

                inertia_old = inertia

            # Calculate consensus for the current run
            current_inertia = d_matrix.min(axis=1).sum()

            # Update best result if current inertia is lower
            if current_inertia < best_inertia:
                best_inertia = current_inertia
                best_centroids = centroids
                best_labels = labels

        self.centroids = best_centroids
        self.labels_ = best_labels
        # self.utility = self.get_utility(self.labels_, self.ls_partitions_labels)
        self.weights = weights
        return self
    
    # def get_utility(self, labels, basic_partitions):
        
    #     ls_utility = []
    #     for pi_i in basic_partitions:
    #         p_k, P_k_i, P_i = self.extract_p(labels, pi_i)
    #         norm_P_k_i = np.linalg.norm(P_k_i, axis=1, ord=2).reshape(-1, 1) ** 2
    #         norm_P_i = np.linalg.norm(P_i, ord=2) ** 2
    #         utility = (p_k * norm_P_k_i).sum() - norm_P_i
    #         ls_utility.append(utility)

    #     return np.mean(ls_utility)
    
    def get_weight_norm(self, X_b_split, centroids_split):
        weight_norm = np.ones((self.r, self.k))
        for i in range(len(X_b_split)):
            P_i = np.unique(self.ls_partitions_labels[i], return_counts=True)[1] # / len(self.ls_partitions_labels[i])
            
            if self.type in 'Uc':            
                weight_norm[i, :] = [np.linalg.norm(centroid, ord=2) ** 2 for centroid in centroids_split[i]]
                weight_norm[i, :] -= np.linalg.norm(P_i, ord=2) ** 2
                weight_norm[i, :] = np.abs(weight_norm[i, :])
            elif self.type == 'Ucos':
                weight_norm[i, :] = [np.linalg.norm(centroid, ord=2) for centroid in centroids_split[i]]
                weight_norm[i, :] -= np.linalg.norm(P_i, ord=2)
                weight_norm[i, :] = np.abs(weight_norm[i, :])
            elif self.type == 'ULp':
                weight_norm[i, :] = [np.linalg.norm(centroid, ord=self.p) for centroid in centroids_split[i]]
                weight_norm[i, :] -= np.linalg.norm(P_i, ord=self.p)
                weight_norm[i, :] = np.abs(weight_norm[i, :])
            elif self.type == 'Uh':
                eps = 1E-10
                entropy_mk_i = np.array([entropy(centroid + eps, base=2) for centroid in centroids_split[i]])
                entropy_P_i = entropy(P_i + eps, base=2)
                weight_norm[i, :] = (- entropy_mk_i) - (- entropy_P_i)
                
        return weight_norm

In [18]:
from src.Dataset import read_dataset, get_binary_dataset

ls_datasets = ['iris'] # 'pendigits', takes too long
d_results = {}
d_time = {}
# for dataset in ls_datasets:
# Load data
dataset = 'iris'
X, y = read_dataset(dataset)
k = len(np.unique(y))
X_b, ls_partitions_labels, cluster_sizes = get_binary_dataset(X, k, r=100)


Clustering Progress: 100%|██████████| 100/100 [00:05<00:00, 19.30it/s]

X_b shape: (150, 768)


In [20]:
model = ConsensusKMeans(n_clusters=len(np.unique(y)), cluster_sizes=cluster_sizes, n_init=10, max_iter=10000, tol=1e-40, type='Uc', normalize=True)
model.fit(X_b).labels_

100%|██████████| 10/10 [00:00<00:00, 19.52it/s]


array([0, 0, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0,
       1, 0, 1, 0, 0, 2, 1, 2, 0, 2, 1, 1, 0, 1, 1, 0, 1, 0, 0, 2, 0, 0,
       1, 0, 0, 0, 2, 1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 0, 2, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 1, 2,
       1, 1, 0, 0, 1, 2, 1, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 0], dtype=int64)

In [62]:
# model = ConsensusKMeans(n_clusters=len(np.unique(y)), n_init=10, max_iter=10000, tol=1e-10, type='Uc', normalize=True)
# model.fit(X).labels_

In [46]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(y.values, model.labels_)

0.7322981167185344

In [6]:
model = ConsensusKMeans(4, random_state=10)
model.fit(X).labels_
X_b = model.get_binary_dataset(X)

# centroids = model.initialize_centroids(X_b)
labels = model.labels_

Clustering Progress: 100%|██████████| 100/100 [00:05<00:00, 19.19it/s]


X_b shape: (150, 812)
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations
Converged in 5 iterations


Clustering Progress: 100%|██████████| 100/100 [00:05<00:00, 19.36it/s]

X_b shape: (150, 763)


In [19]:
X_b_split = np.split(X_b, np.cumsum(model.cluster_sizes), axis=1)[:-1]
centroids_split = np.split(model.centroids, np.cumsum(model.cluster_sizes), axis=1)[:-1]
X_r = [cdist(X_b_split[i], centroids_split[i], 'euclidean') for i in range(len(X_b_split))]

# X_r = []
if True:
    weight_norm = np.ones((model.r, model.k))
    for i in range(len(X_b_split)):
        _, _, P_i = model.extract_p(labels, model.ls_partitions_labels[i])
        X_r_i = np.zeros((len(X_b_split[i]), model.k))
        weight_norm[i, :] = [np.linalg.norm(centroid, ord=2) ** 2 for centroid in centroids_split[i]] 
        weight_norm[i, :] -= np.linalg.norm(P_i, ord=2) ** 2
        # for j, centroid in enumerate(centroids_split[i]):
            # weight_factor = np.linalg.norm(centroid, ord=2) ** 2 - np.linalg.norm(P_i, ord=2) ** 2
            # d = cdist(X_b_split[i], centroid.reshape(1, -1), 'euclidean') ** 2
            # X_r_i[:, j] = d.ravel()
            # X_r_i[:, j] = X_r_i/ weight_factor
        # X_r.append(X_r_i)



In [25]:
np.linalg.norm(P_i, ord=2) ** 2

0.26257777777777785

In [23]:
X_r[0]

array([[0.07443229, 0.89285714, 1.13648581, 1.24227718],
       [1.37850535, 1.00572342, 0.46      , 1.24227718],
       [1.33978127, 1.00572342, 1.13648581, 0.4991342 ],
       [1.37850535, 1.00572342, 0.46      , 1.24227718],
       [0.07443229, 0.89285714, 1.13648581, 1.24227718],
       [1.37850535, 1.00572342, 0.46      , 1.24227718],
       [1.37850535, 1.00572342, 1.13648581, 0.91507936],
       [0.07443229, 0.89285714, 1.13648581, 1.24227718],
       [1.37850535, 1.00572342, 0.46      , 1.24227718],
       [1.33978127, 1.00572342, 1.13648581, 0.4991342 ],
       [1.37850535, 1.00572342, 0.46      , 1.24227718],
       [1.37850535, 1.00572342, 1.13648581, 0.91507936],
       [0.07443229, 0.89285714, 1.13648581, 1.24227718],
       [0.07443229, 0.89285714, 1.13648581, 1.24227718],
       [1.37850535, 1.00572342, 0.46      , 1.24227718],
       [1.37850535, 1.00572342, 1.13648581, 0.91507936],
       [1.33978127, 1.00572342, 1.13648581, 0.4991342 ],
       [0.07443229, 0.89285714,

In [20]:
weight_norm

array([[ 6.37699231e-01, -2.51098186e-01,  2.90222222e-02,
         2.80674817e-01],
       [ 3.80140597e-01,  1.67594943e+00,  5.91155556e-01,
         4.63530642e-01],
       [-1.56988612e-02,  2.77170295e-01,  3.43111111e-02,
        -1.52147020e-01],
       [-2.38488889e-01, -1.86192971e-01, -1.08888889e-01,
         1.85386544e-01],
       [ 4.95129825e-01,  2.77621769e-01,  2.37066667e-01,
         1.26890888e-01],
       [-2.38488889e-01, -1.92570522e-01, -2.68888889e-02,
        -1.20841830e-01],
       [-1.41530809e-01,  2.45051247e-01, -2.86222222e-02,
         1.55300269e-02],
       [ 6.69911727e-01,  8.90133333e-01,  4.12933333e-01,
         8.52071050e-01],
       [-2.62577778e-01,  5.34616100e-01,  2.40622222e-01,
        -1.38010304e-01],
       [ 1.66115137e+00,  7.75124036e-01,  7.63644444e-01,
         6.52917801e-01],
       [ 6.13228440e-01,  5.50731973e-01,  5.58266667e-01,
         4.72241753e-01],
       [ 9.34527547e-02,  3.39403628e-01,  2.80444444e-02,
      

In [18]:
[np.linalg.norm(centroid, ord=2) ** 2 for centroid in centroids_split[i]]

[0.9002770083102494, 0.011479591836734696, 0.2916, 0.5432525951557091]

In [16]:
len(X_r)

100

In [12]:
cdist(X_b_split[i], centroid.reshape(1, -1), 'euclidean')

array([[0.07443229],
       [1.37850535],
       [1.33978127],
       [1.37850535],
       [0.07443229],
       [1.37850535],
       [1.37850535],
       [0.07443229],
       [1.37850535],
       [1.33978127],
       [1.37850535],
       [1.37850535],
       [0.07443229],
       [0.07443229],
       [1.37850535],
       [1.37850535],
       [1.33978127],
       [0.07443229],
       [0.07443229],
       [1.33978127],
       [1.37850535],
       [1.37850535],
       [1.37850535],
       [1.37850535],
       [1.33978127],
       [1.37850535],
       [0.07443229],
       [1.33978127],
       [1.33978127],
       [1.37850535],
       [0.07443229],
       [1.33978127],
       [1.37850535],
       [0.07443229],
       [1.37850535],
       [0.07443229],
       [1.37850535],
       [0.07443229],
       [1.33978127],
       [1.33978127],
       [1.33978127],
       [1.37850535],
       [1.33978127],
       [1.33978127],
       [0.07443229],
       [1.37850535],
       [1.33978127],
       [1.378

In [49]:
X_b_split[i]

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],


In [11]:
(cdist(X_b_split[i], centroid.reshape(1, -1), 'euclidean') ** 2).shape

(150, 1)

array([1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484444,
       1.47484444, 1.47484444, 1.47484444, 1.47484444, 1.47484

In [28]:
centroid.reshape(1, -1)

array([[0., 0., 0., 1.]])

In [20]:
np.linalg.norm(X_b_split[i] - centroid, axis=1, ord=2).shape

(150,)

In [7]:
X_r

[]

In [10]:
X_b_split = np.split(X_b, np.cumsum(model.cluster_sizes), axis=1)[:-1]
centroids_split = np.split(centroids, np.cumsum(model.cluster_sizes), axis=1)[:-1]
X_b_split

[array([[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 arr

In [19]:
X_b_split[0], centroids_split[0]

(array([[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [30]:
X_b_split[0]

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
centroids_split[0]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [54]:
d_mat = [cdist(X_b_split[i], centroids_split[i], metric='jensenshannon') for i in range(len(X_b_split))]
d_mat = np.sum(d_mat, axis=0)
d_mat = d_mat / 100
print(d_mat.min(axis=1).sum())
d_mat.min(axis=1)

74.03075602414248


array([0.82422907, 0.83255461, 0.54948604, 0.        , 0.70767142,
       0.69934587, 0.12488319, 0.54948604, 0.03330218, 0.09158101,
       0.07492992, 0.09158101, 0.83255461, 0.52450941, 0.09990655,
       0.09158101, 0.18316201, 0.31637075, 0.74929915, 0.69934587,
       0.59943932, 0.19148756, 0.        , 0.23311529, 0.04162773,
       0.48288167, 0.83255461, 0.83255461, 0.74929915, 0.70767142,
       0.64106705, 0.33302184, 0.82422907, 0.50785831, 0.54948604,
       0.16651092, 0.83255461, 0.82422907, 0.83255461, 0.49953277,
       0.62441596, 0.34967294, 0.70767142, 0.83255461, 0.41627731,
       0.6327415 , 0.79925243, 0.09158101, 0.36632403, 0.41627731,
       0.83255461, 0.28306857, 0.50785831, 0.64106705, 0.70767142,
       0.74929915, 0.70767142, 0.12488319, 0.60776487, 0.69934587,
       0.64106705, 0.41627731, 0.83255461, 0.09158101, 0.69934587,
       0.73264806, 0.70767142, 0.12488319, 0.83255461, 0.29971966,
       0.41627731, 0.64106705, 0.44957949, 0.        , 0.69934

In [38]:
centroids_split[0][-1]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [34]:
X_b_split[0][2]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
X_out = np.zeros((X_b.shape[0], 4))
for i in range(len(X_b_split[0])):
    for j in range(4):
        X_out[:, j] = np.sum((X_b_split[0][i] - centroids_split[0][j]) ** 2)


X_out

array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       ...,
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])

In [62]:
# Split each row by the number of clusters in each run
def f(X_b, centroids, cluster_sizes, weights=None):
    X_b_split = np.split(X_b, np.cumsum(cluster_sizes), axis=1)[:-1]
    centroids_split = np.split(centroids, np.cumsum(cluster_sizes), axis=1)[:-1]
    X_r = [cdist(X_b_split[i], centroids_split[i], metric='euclidean') for i in range(len(X_b_split))]
    if weights is None:
        return np.average(X_r, axis=0)
    else:
        return np.average(X_r, weights=weights, axis=0)

weights = np.ones(100)
weights[0] = 10
f(X_b, centroids, cluster_sizes, weights=None)

array([[0.94752309, 0.48083261, 1.41421356, 1.41421356],
       [0.46669048, 0.94752309, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356, 1.41421356],
       [0.94752309, 0.94752309, 0.94752309, 0.46669048],
       [1.41421356, 1.41421356, 0.94752309, 0.94752309],
       [0.48083261, 0.48083261, 1.41421356, 1.41421356],
       [0.94752309, 0.48083261, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 0.94752309, 0.94752309],
       [0.94752309, 0.48083261, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356, 1.41421356],
       [0.48083261, 0.48083261, 1.41421356, 1.41421356],
       [0.48083261, 0.48083261, 1.41421356, 1.41421356],
       [0.48083261, 0.48083261, 1.41421356, 1.41421356],
       [0.46669048, 0.94752309, 1.41421356, 1.41421356],
       [0.94752309, 1.41421356, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356, 1.41421356],
       [0.94752309, 0.48083261,

In [34]:
cdist(np.split(X_b[[0]], np.cumsum(cluster_sizes), axis=1)

[array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1.]]),
 array([[0.,

In [16]:
model.inertia

460.1359737991966

In [10]:
from sklearn.cluster import KMeans as sk_KMeans
from sklearn.metrics.cluster import rand_score
model_sk = sk_KMeans(4, random_state=10)
model_sk.fit(X)
rand_score(model_sk.labels_, model.labels)

c:\Users\alber\Documents\GitHub\KCC_K-Means_Consensus_Clustering\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.7857718120805369

In [11]:
model.centroids

array([[ 0.1757777 , -1.0855001 ,  0.23123741,  0.03086579],
       [-0.85603136,  0.66278386, -1.02116825, -0.99682228],
       [ 1.07150731,  0.18610456,  0.92214282,  0.95107504],
       [-0.24072152, -1.12086341,  0.39521949,  0.42629726]])

In [12]:
model = KMeans(4)
cents = model.initialize_centroids(X.values)
cents

array([[ 6.72249049e-01,  3.36720285e-01,  8.73563532e-01,
         1.44312105e+00],
       [-4.14620671e-01, -1.04706171e+00,  3.63481020e-01,
         1.74711992e-03],
       [ 6.72249049e-01,  3.36720285e-01,  8.73563532e-01,
         1.44312105e+00],
       [ 4.30722444e-01, -1.96958304e+00,  4.20156854e-01,
         3.94849102e-01]])

In [13]:
import numpy_indexed as npi
arr = np.array([[1, 1, 1], [0, 1, 2], [1, 2, 3]])
npi.group_by([0, 1, 1]).sum(arr[:, 1:])

(array([0, 1]),
 array([[1, 1],
        [3, 5]]))

In [48]:
model.update_centroids(X, d_matrix.argmin(axis=1))

(array([0, 1, 2, 3], dtype=int64),
 array([[-1.08430807, -0.51591306, -0.85590815, -0.87181284],
        [-0.38644257,  0.50200536, -0.36952644, -0.29526327],
        [ 1.96039094,  0.83641934,  1.42142993,  1.39944306],
        [ 0.92561519, -0.35517071,  0.82133286,  0.74684238]]))

In [28]:
cdist([[1, 0]], [[0, 1]])

array([[1.41421356]])

In [34]:
from scipy.spatial.distance import cdist
arr = np.zeros(X.shape[0])
arr[0] = 1
d_matrix = cdist(X, cents)
d_matrix.argmin(axis=1)

array([3, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 0, 1, 0, 1, 3, 3, 0, 1, 3, 0, 1,
       1, 3, 2, 3, 3, 3, 0, 0, 1, 1, 3, 0, 0, 3, 3, 1, 1, 1, 1, 1, 3, 1,
       1, 0, 3, 3, 3, 2, 1, 2, 1, 1, 3, 1, 1, 1, 1, 0, 3, 1, 0, 1, 1, 1,
       1, 2, 0, 3, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 3, 0, 0, 0, 3, 0, 1, 1,
       1, 1, 0, 3, 1, 3, 2, 0, 0, 0, 3, 3, 0, 1, 0, 3, 1, 0, 3, 1, 0, 3,
       3, 3, 3, 3, 1, 3, 1, 3, 0, 1, 1, 0, 3, 1, 1, 1, 3, 3, 1, 0, 2, 3,
       0, 3, 0, 3, 3, 3, 1, 3, 3, 3, 1, 1, 1, 3, 0, 1, 1, 3], dtype=int64)